In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import numpy as np
import mapping
from tensorflow.keras.preprocessing import image
import splitfolders
import matplotlib.pyplot as plt
from tensorflow.keras.utils import register_keras_serializable
from custom_layers.MinPooling import MinPooling2D

In [3]:
label_map_marital_status={
    "single_m": "أعزب",
    "single_f": "أنسة",
    "married_m": "متزوج",
    "married_f": "متزوجة",
    "widow": "أرملة",
    "widower": "أرمل",
}

In [4]:
img_size = (64, 64)
batch_size = 32

Marital_status_train_ds = tf.keras.utils.image_dataset_from_directory(
    r"..\data\dataset_cnn_2\data\Marital status\train",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)

Marital_status_val_ds = tf.keras.utils.image_dataset_from_directory(
    r"..\data\dataset_cnn_2\data\Marital status\val",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)

Marital_status_test_ds = tf.keras.utils.image_dataset_from_directory(
    r"..\data\dataset_cnn_2\data\Marital status\test",
    image_size=img_size,
    batch_size=batch_size,
    label_mode="categorical"
)


normalization_layer = tf.keras.layers.Rescaling(1./255)

class_names_Marital_status = Marital_status_train_ds.class_names
print("Classes:", class_names_Marital_status)
num_classes = len(class_names_Marital_status)

Marital_status_train_ds = Marital_status_train_ds.map(lambda x, y: (normalization_layer(x), y))
Marital_status_val_ds = Marital_status_val_ds.map(lambda x, y: (normalization_layer(x), y))
Marital_status_test_ds = Marital_status_test_ds.map(lambda x, y: (normalization_layer(x), y))

Found 2895 files belonging to 6 classes.
Found 783 files belonging to 6 classes.
Found 459 files belonging to 6 classes.
Classes: ['married_f', 'married_m', 'single_f', 'single_m', 'widow', 'widower']


In [4]:
model = models.Sequential([
    #layers.Conv2D(16, 3, activation='relu', input_shape=(64, 64, 3)),
    #layers.MaxPooling2D(),
    
    layers.Conv2D(32, 3, activation='relu'),
    MinPooling2D(pool_size=(2,2)),
    #layers.MaxPooling2D(),
    #layers.AveragePooling2D(pool_size=(2,2)),

    layers.Conv2D(64, 3, activation='relu'),
    #layers.MaxPooling2D(),
    
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(256, 3, activation='relu'),
    #layers.AveragePooling2D(pool_size=(2,2)),
    MinPooling2D(pool_size=(2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
history = model.fit(Marital_status_train_ds, epochs=35, 
                    validation_data=Marital_status_val_ds)

Epoch 1/35

91/91 ━━━━━━━━━━━━━━━━━━━━ 97s 632ms/step - accuracy: 0.7468 - loss: 0.6540 - val_accuracy: 0.9962 - val_loss: 0.0138
Epoch 2/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 56s 612ms/step - accuracy: 0.9903 - loss: 0.0359 - val_accuracy: 0.9949 - val_loss: 0.0187
Epoch 3/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 88s 673ms/step - accuracy: 0.9955 - loss: 0.0102 - val_accuracy: 0.9974 - val_loss: 0.0044
Epoch 4/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 58s 627ms/step - accuracy: 0.9959 - loss: 0.0095 - val_accuracy: 0.9962 - val_loss: 0.0159
Epoch 5/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 57s 621ms/step - accuracy: 0.9990 - loss: 0.0036 - val_accuracy: 0.9974 - val_loss: 0.0057
Epoch 6/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 57s 621ms/step - accuracy: 1.0000 - loss: 1.5765e-04 - val_accuracy: 1.0000 - val_loss: 5.1846e-04
Epoch 7/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 55s 595ms/step - accuracy: 1.0000 - loss: 2.6288e-05 - val_accuracy: 1.0000 - val_loss: 7.4275e-04
Epoch 8/35
91/91 ━━━━━━━━━━━━━━━━━━━━ 54s 590ms/step - accuracy: 1.0000 - loss: 

In [7]:
test_loss, test_acc = model.evaluate(Marital_status_test_ds)

15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.9978 - loss: 0.0054


In [23]:
folder_path = r""
#D:\ocr_project\data\dataset_cnn_2\output_img\result\crops\gender\male
#religion\muslim_m   "D:\ocr_project\data\dataset_cnn_2\output_img\result\crops\gender\male"
#marital status\widower   "..\data\dataset_cnn_2\data\test\male"
output_file = "../outputs/predictions_CNN_muslim.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)

       
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

      
        img = image.load_img(img_path, target_size=(64,64))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0

     
        predictions = model.predict(img_array, verbose=0)
        predicted_label = np.argmax(predictions, axis=1)[0]

        en_label = class_names_Marital_status[predicted_label]
        ar_label = label_map_marital_status.get(en_label, en_label)

     
        f.write(f"{img_name}: {ar_label}\n")

print(f"{output_file}")

../outputs/predictions_CNN_muslim.txt


In [10]:
model.save("../models/cnn_model_marital_status.keras")

In [6]:
model = models.load_model("../models/cnn_model_marital_status.keras")